In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('VA_post_dig_test.csv')

In [3]:
def extract_street_addresses(dataframe, address_column):
    street_addresses = []  #Initialize an empty list to store extracted street addresses

    #Traverse the rows of the dataframe
    for index, row in dataframe.iterrows():
        full_address = row['FULL_ADDRESS'] 

        #Extract the desired part of the address using string manipulation
        #Split the full address by comma and take the first part as the street address
        street_address = full_address.split(',')[0].strip()

        #Add the street address to the list
        street_addresses.append(street_address)

    #Create a new dataframe with the extracted street addresses
    extracted_addresses_df = pd.DataFrame({
        'Extracted Street Addresses': street_addresses,
    })
    return extracted_addresses_df

In [4]:
def extract_addresses_with_vlookup_one(dataframe, vlookup_column, address_column):
    
    filtered_df = dataframe[dataframe[vlookup_column] == 1]
    result_df = filtered_df[[address_column]].copy()

    # Extract the indices of the selected rows as a list
    index_list = filtered_df.index.tolist()

    # Add a new 'Index' column to the result DataFrame
    result_df['Index'] = index_list

    return result_df

In [5]:
#build df with vlookup addresses
checkInc = extract_addresses_with_vlookup_one(df, vlookup_column='Lookup', address_column='FULL_ADDRESS')
#make a copy of it
index_df = checkInc.copy()
#drop address to get df with just indexes of vlookups
index_df.drop('FULL_ADDRESS', axis=1, inplace=True)
#extract the only street address from df
checkInc = extract_street_addresses(checkInc, 'FULL_ADDRESS')

df_Check = checkInc.join(index_df)
df_Check.drop('Extracted Street Addresses', axis=1, inplace=True)
df_Full = extract_street_addresses(df, 'FULL_ADDRESS')
checkIndex = df_Check['Index'].tolist()

print(checkIndex)
print(df_Full)

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 87.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
     Extracted Street Addresses
0                     0 COAL RD
1                 0 NEW SERVICE
2                 0 NEW SERVICE
3                 0 NEW SERVICE
4               0 PATTERSON TRL
...             

In [6]:
df_Full['Incremental'] = np.nan

for i in checkIndex:
    address_Main = df_Full.loc[i,'Extracted Street Addresses']
    address_Above = df_Full.loc[i-1,'Extracted Street Addresses']
    address_Below = df_Full.loc[i+1,'Extracted Street Addresses']
    
    if(address_Main == address_Below or address_Main == address_Above):
        df_Full.loc[i,'Incremental'] = 0
    else:
        df_Full.loc[i,'Incremental'] = 1


KeyError: nan

In [ ]:
print(df_Full)
df.drop('Incremental', axis = 1, inplace = True)
merged_df = df.join(df_Full)
merged_df.drop('Extracted Street Addresses',axis = 1, inplace = True)

In [ ]:
#Save DataFrame to CSV file

merged_df.to_csv('VA_post_dig_test_output.csv.csv', index=False)
